### SP025 Integration analysis Integration TME part1

In [29]:
import scanpy as sc
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import rcParams
from matplotlib import colors
import seaborn as sb
#from gprofiler import GProfiler
#import loompy as lp
#import rpy2.rinterface_lib.callbacks
import logging
#import scrublet as scr
#from rpy2.robjects import pandas2ri
#import anndata2ri
import warnings
warnings.filterwarnings('ignore')

In [30]:
#sc.logging.print_header()

In [31]:
import os              
os.environ['PYTHONHASHSEED'] = '0'
import desc
#import keras
import tensorflow as tf
from time import time                                                       
import sys

### Import dataset Lung/BC/CRC/Ovarian CD8 + CD4

In [32]:
adataLung = sc.read_h5ad("/home/spuccio/isilon/spuccio/SP025_NaClTcell/Analysis/Lung_output/adata_lung_Tcell.h5ad")
adataBC = sc.read_h5ad("/home/spuccio/isilon/spuccio/SP025_NaClTcell/Analysis/BC_output/adata_bc_Tcell.h5ad")
adataCRC = sc.read_h5ad("/home/spuccio/isilon/spuccio/SP025_NaClTcell/Analysis/CRC_output/adata_CRC_Tcell_sub.h5ad")
adataOvarian = sc.read_h5ad("/home/spuccio/isilon/spuccio/SP025_NaClTcell/Analysis/Ovarian_output/adata_oc_Tcell_sub.h5ad")

In [33]:
adata = adataLung.concatenate(adataBC,adataCRC,adataOvarian,index_unique=None)

In [34]:
adata.obs = adata.obs[['CellFromTumor','PatientNumber','TumorType']].astype('category')

In [35]:
adata = adata[adata.obs['CellFromTumor']  == 1,:]

In [36]:
adata.write("/home/spuccio/isilon/spuccio/SP025_NaClTcell/Analysis/Integrated/Concatenate_T_cell.h5ad")

In [37]:
print(adata.X.shape)

(31018, 11358)


### Execute DESC algorithm 

In [38]:
adata2 = adata.copy()

sc.pp.normalize_per_cell(adata2, counts_per_cell_after=1e4)
sc.pp.log1p(adata2)

In [39]:
adata2=desc.scale_bygroup(adata2,"TumorType")

In [40]:
save_dir="/home/spuccio/isilon/spuccio/SP025_NaClTcell/Analysis/Integrated"
adata2=desc.train(adata2,
        dims=[adata.shape[1],64,32],
        tol=0.001,
        n_neighbors=10,
        batch_size=256,
        louvain_resolution=[0.4,0.5,0.6,0.7,0.8],# not necessarily a list, you can only set one value, like, louvain_resolution=1.0
        save_dir=str(save_dir),
        do_tsne=False,
        learning_rate=200, # the parameter of tsne
        use_GPU=False,
        num_Cores=30, #for reproducible, only use 1 cpu
        num_Cores_tsne=4,
        save_encoder_weights=False,
        save_encoder_step=3,# save_encoder_weights is False, this parameter is not used
        use_ae_weights=False,
        do_umap=True)

Start to process resolution= 0.4
The number of cpu in your computer is 32
Failed to import pydot. You must install pydot and graphviz for `pydotprint` to work.
use_ae_weights=False, the program will rerun autoencoder
Pretraining the 1th layer...
learning rate = 0.1
Train on 31018 samples
Epoch 1/50
31018/31018 [==============================] - 17s 542us/sample - loss: 0.7331
Epoch 2/50
31018/31018 [==============================] - 16s 518us/sample - loss: 0.7300
Epoch 3/50
31018/31018 [==============================] - 16s 510us/sample - loss: 0.7284
Epoch 4/50
31018/31018 [==============================] - 17s 546us/sample - loss: 0.7271
Epoch 5/50
31018/31018 [==============================] - 19s 612us/sample - loss: 0.7260
Epoch 6/50
31018/31018 [==============================] - 19s 611us/sample - loss: 0.7245
Epoch 7/50
31018/31018 [==============================] - 19s 622us/sample - loss: 0.7228
Epoch 8/50
31018/31018 [==============================] - 19s 620us/sample - loss

In [41]:
adata2.write("/home/spuccio/isilon/spuccio/SP025_NaClTcell/Analysis/Integrated/Desc_T_cell.h5ad")

In [42]:
adata2

AnnData object with n_obs × n_vars = 31018 × 11358
    obs: 'CellFromTumor', 'PatientNumber', 'TumorType', 'n_counts', 'desc_0.4', 'desc_0.5', 'desc_0.6', 'desc_0.7', 'desc_0.8'
    var: 'gene_id', 'mt', 'rb', 'gene_biotype', 'n_cells-0', 'n_cells_by_counts-0', 'mean_counts-0', 'log1p_mean_counts-0', 'pct_dropout_by_counts-0', 'total_counts-0', 'log1p_total_counts-0', 'highly_variable-0', 'means-0', 'dispersions-0', 'dispersions_norm-0', 'n_cells-1', 'n_cells_by_counts-1', 'mean_counts-1', 'log1p_mean_counts-1', 'pct_dropout_by_counts-1', 'total_counts-1', 'log1p_total_counts-1', 'highly_variable-1', 'means-1', 'dispersions-1', 'dispersions_norm-1', 'n_cells-2', 'n_cells_by_counts-2', 'mean_counts-2', 'log1p_mean_counts-2', 'pct_dropout_by_counts-2', 'total_counts-2', 'log1p_total_counts-2', 'highly_variable-2', 'means-2', 'dispersions-2', 'dispersions_norm-2', 'n_cells-3', 'n_cells_by_counts-3', 'mean_counts-3', 'log1p_mean_counts-3', 'pct_dropout_by_counts-3', 'total_counts-3', 'log1